# Ejercicios

In [0]:
%sql
create catalog if not exists databrick_intro;
create schema if not exists databrick_intro.sesion2;
create volume if not exists databrick_intro.sesion2.landing;

In [0]:
%sh
curl -L https://raw.githubusercontent.com/jmartinezceste/Databricks_master/refs/heads/main/sesion1/data/spark_intro.csv -o /Volumes/databrick_intro/sesion2/landing/spark_intro.csv

%sh
curl -L https://raw.githubusercontent.com/jmartinezceste/Databricks_master/refs/heads/main/sesion1/data/dim_spark_intro.csv -o /Volumes/databrick_intro/sesion2/landing/dim_spark_intro.csv

In [0]:
from pyspark.sql import functions as F

# Read CSV from volume as a Spark DataFrame
sales_df = (
    spark.read
    .option("header", True)
    .option("inferSchema", True)  # Let Spark infer column types for now
    .csv("/Volumes/databrick_intro/sesion2/landing/spark_intro.csv")
)

display(sales_df)

In [0]:
sales_df.printSchema()

### Creación de df potente

In [0]:
data = [
    (1, "Ana", "Madrid", "IT", 34, 5, 3000),
    (2, "Luis", "Barcelona", "Ventas", 28, 2, 2500),
    (3, "María", "Madrid", "IT", 45, 10, 4000),
    (4, "Juan", "Sevilla", "Marketing", 30, 3, 2800),
    (5, "Laura", "Barcelona", "IT", 38, 7, 3500),
    (6, "Carlos", "Madrid", "Ventas", 41, 12, 4200),
    (7, "Elena", "Valencia", "Marketing", 29, 1, 2300),
    (8, "Pedro", "Sevilla", "IT", 36, 6, 3200),
    (9, "Sofía", "Madrid", "RRHH", 33, 4, 2900),
    (10, "Jorge", "Barcelona", "Ventas", 50, 20, 5000)
]

columns = [
    "id",
    "nombre",
    "ciudad",
    "departamento",
    "edad",
    "antiguedad_anios",
    "salario_mensual"
]

df = spark.createDataFrame(data, columns)
display(df)


## Select

In [0]:
# Tu codigo
# Select a subset of columns
sales_simple_df = sales_df.select("order_id", "order_date", "country", "units_sold")
display(sales_simple_df)

### Ejercicio
Selecciona las columnas nombre, edad y salario.

In [0]:
# Tu codigo

## filter

In [0]:
from pyspark.sql.functions import col
# Tu codigo
# Filter sales only for Spain
spain_sales_df = sales_df.filter(col("country") == "Spain")
display(spain_sales_df)

# Filter sales with more than 3 units sold
big_orders_df = sales_df.filter(col("units_sold") > 3)
display(big_orders_df)

### Ejercicio
Filtra las personas:
- Mayores de 35 años
- Que vivan en Madrid

In [0]:
# Tu codigo

## withColumn()

In [0]:
# Tu codigo
# Create a new column with total sales amount
sales_with_total_df = sales_df.withColumn(
    "total_sales",
    col("units_sold") * col("unit_price")  # Simple numeric expression
)

display(sales_with_total_df)

In [0]:
from pyspark.sql.functions import (
    col,
    when,
    year,
    round
)

sales_enriched_df = (
    sales_df
    # 1. Total de la venta
    .withColumn(
        "total_sales",
        col("units_sold") * col("unit_price")
    )
    # 2. Descuento según categoría
    .withColumn(
        "discount_rate",
        when(col("category") == "Electronics", 0.10)
        .when(col("category") == "Clothing", 0.20)
        .when(col("category") == "Food", 0.05)
        .otherwise(0.0)
    )
    # 3. Venta final tras descuento
    .withColumn(
        "final_sales",
        round(
            col("total_sales") * (1 - col("discount_rate")),
            2
        )
    )
    # 4. Año del pedido
    .withColumn(
        "order_year",
        year(col("order_date"))
    )
    # 5. Flag de pedido de alto valor
    .withColumn(
        "high_value_order",
        col("final_sales") > 1000
    )
)

display(sales_enriched_df)


### Ejercicio
Crea una columna salario_mensual_k que sea el salario en miles de euros.

In [0]:
# Tu codigo
df_with_salario_k = df.withColumn("salario_mensual_k ", col("salario_mensual") / 1000)
display(df_with_salario_k)

## groupby()

In [0]:
# Tu codigo
# Aggregate total units and total sales by country
sales_country_df = (
    sales_with_total_df
    .groupBy("country")
    .agg(
        F.sum("units_sold").alias("total_units"),
        F.sum("total_sales").alias("total_sales_amount"),
        F.avg("total_sales").alias("avg_sales_amount"),
        F.max("total_sales").alias("max_sales_amount")
    )
)

display(sales_country_df)

In [0]:
# Aggregate total units and total sales by country
sales_country_df = (
    sales_with_total_df
    .groupBy("country").sum("total_sales")
)
display(sales_country_df)

### Ejercicio
Calcula por ciudad:
- Salario medio
- Edad media

In [0]:
# Tu codigo
from pyspark.sql import functions as F

ciudad_stats_df = (
    df
    .groupBy("ciudad")
    .agg(
        F.avg("salario_mensual").alias("salario_medio"),
        F.avg("edad").alias("edad_media")
    )
)

display(ciudad_stats_df)

In [0]:
# Tu codigo

## orderBy()

In [0]:
# Tu codigo

### Ejercicio
Ordena las personas por edad de mayor a menor.

In [0]:
# Tu codigo

### Ejercicio bonus
Obtén la ciudad con el salario medio más alto.

## join()


In [0]:
# Tu codigo

### Ejercicio
Aquí vamos a añadir una nueva tabla.

Disponemos de una segunda tabla con información de proyectos de la empresa.
Cada proyecto pertenece a un departamento y tiene un presupuesto anual.

El objetivo es combinar la información de empleados con la de proyectos usando un join.

In [0]:
data_proyectos = [
    (101, "Plataforma Web", "IT", 120000),
    (102, "App Móvil", "IT", 90000),
    (103, "Campaña Verano", "Marketing", 60000),
    (104, "Expansión Norte", "Ventas", 150000),
    (105, "Optimización Procesos", "RRHH", 40000)
]

columns_proyectos = [
    "id_proyecto",
    "nombre_proyecto",
    "departamento",
    "presupuesto_anual"
]

df_proyectos = spark.createDataFrame(data_proyectos, columns_proyectos)
df_proyectos.show()


👉 Realiza un join entre:
- df (empleados)
- df_proyectos (proyectos)
usando la columna departamento.

A partir del DataFrame resultante:
- Muestra el nombre del empleado, su departamento y el nombre del proyecto
- Calcula el salario medio por proyecto
- Ordena el resultado por presupuesto anual descendente

In [0]:
# Tu codigo

## write()

In [0]:
# Tu codigo

### Ejercicio
👉 A partir del DataFrame de empleados:

1. Filtra los empleados del departamento IT
2. Selecciona:
    - nombre
    - ciudad
    - salario_mensual
3. Escribe el resultado en:
    - Formato Parquet
    - Modo overwrite
    - Ruta: "/tmp/empleados_it"

(No es necesario ejecutar realmente la escritura si el entorno no lo permite, lo importante es el código)

In [0]:
# Tu codigo